<a href="https://colab.research.google.com/github/yweslakarep123/fake_news/blob/main/fake_news_detection_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [5]:
!chmod 600 /content/kaggle.json

In [6]:
!kaggle competitions download -c fake-news

 54% 25.0M/46.5M [00:01<00:01, 19.1MB/s]
100% 46.5M/46.5M [00:01<00:00, 33.5MB/s]


In [7]:
!pip install patool pyunpack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 3.6 MB/s eta 0:00:00


In [8]:
from pyunpack import Archive
zip_file_path = '/content/fake-news.zip'
extract_directory = '/content/drive/MyDrive/Test_dataset'
# Buka dan ekstrak file ZIP menggunakan patool
Archive(zip_file_path).extractall(extract_directory)

# Output pesan jika ekstraksi selesai
print("Ekstraksi selesai.")

Ekstraksi selesai.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [20]:
!mkdir fake-news

In [11]:
!mv fake-news Test_dataset

In [12]:
!pwd

/content


In [13]:
%cd Test_dataset

/content/Test_dataset


In [17]:
%cd ..


/content


In [4]:
%cd /content/drive/MyDrive/Test_dataset

/content/drive/MyDrive/Test_dataset


In [19]:
!ls

brain_tumor_dataset			     no			  UpdatedResumeDataSet.csv
dogs_vs_cats				     submit.csv		  yes
house-prices-advanced-regression-techniques  test_fake_news.csv   ZOMATO_DATASET
Mall_Customers.csv			     train_fake_news.csv


In [21]:
!mv train_fake_news.csv test_fake_news.csv submit.csv fake-news/

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Test_dataset/fake-news/train_fake_news.csv')

In [23]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [25]:
df.shape

(20800, 5)

In [26]:
df['title'].unique()

array(['House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It',
       'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart',
       'Why the Truth Might Get You Fired', ...,
       'N.F.L. Playoffs: Schedule, Matchups and Odds - The New York Times',
       'Macy’s Is Said to Receive Takeover Approach by Hudson’s Bay - The New York Times',
       'NATO, Russia To Hold Parallel Exercises In Balkans'], dtype=object)

In [28]:
df['title'].isnull().sum()

558

In [29]:
20800-20242

558

In [6]:
df = df.dropna()

In [31]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [7]:
x = df.drop(['label'],axis=1)

In [8]:
x.shape

(18285, 4)

In [9]:
y = df['label']

In [10]:
y.shape

(18285,)

In [39]:
import tensorflow as tf


In [11]:
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot

In [12]:
vocab_size = 5000

In [13]:
new_x = x.copy()

In [14]:
new_x.reset_index(inplace=True)

In [46]:
new_x.shape

(18285, 5)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
from nltk.stem.porter import PorterStemmer

In [18]:
stemmer = PorterStemmer()

In [19]:
corpus = []

for i in range(0,len(new_x)):
  review = re.sub('[^a-zA-Z]', ' ', new_x['title'][i])
  review = review.lower()
  review = review.split()

  review = [stemmer.stem(kata) for kata in review if not kata in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

In [20]:
one_hot_corpus = [one_hot(words,vocab_size) for words in corpus]

In [78]:
len(one_hot_corpus)

18285

In [21]:
length = 20
padded_corpus = pad_sequences(one_hot_corpus, padding='pre', maxlen=length)

In [80]:
padded_corpus[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2789,
       2643,  479, 3657, 3229, 3347, 3313,  995, 4027, 3497], dtype=int32)

In [55]:
vec_feature = 40

model = Sequential()

model.add(Embedding(vocab_size, vec_feature, input_length=length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))

In [56]:
model.compile(loss= 'binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 64)                26880     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 226945 (886.50 KB)
Trainable params: 226945 (886.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [24]:
data_x = np.array(padded_corpus)
data_y = np.array(y)

In [85]:
data_x.shape, data_y.shape

((18285, 20), (18285,))

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
x_train,x_test,y_train,y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42, shuffle=True)

In [57]:
model.fit(x_train,y_train, validation_data=(x_test,y_test), epochs=10, batch_size=32)

Epoch 1/10
458/458 [==============================] - 13s 24ms/step - loss: 0.2680 - accuracy: 0.8823 - val_loss: 0.2026 - val_accuracy: 0.9119
Epoch 2/10
458/458 [==============================] - 3s 7ms/step - loss: 0.1527 - accuracy: 0.9398 - val_loss: 0.2000 - val_accuracy: 0.9166
Epoch 3/10
458/458 [==============================] - 4s 8ms/step - loss: 0.1335 - accuracy: 0.9495 - val_loss: 0.2034 - val_accuracy: 0.9166
Epoch 4/10
458/458 [==============================] - 4s 8ms/step - loss: 0.1228 - accuracy: 0.9545 - val_loss: 0.1987 - val_accuracy: 0.9215
Epoch 5/10
458/458 [==============================] - 3s 6ms/step - loss: 0.1134 - accuracy: 0.9578 - val_loss: 0.2036 - val_accuracy: 0.9191
Epoch 6/10
458/458 [==============================] - 3s 6ms/step - loss: 0.1078 - accuracy: 0.9616 - val_loss: 0.2071 - val_accuracy: 0.9204
Epoch 7/10
458/458 [==============================] - 3s 6ms/step - loss: 0.1003 - accuracy: 0.9651 - val_loss: 0.2236 - val_accuracy: 0.9150
Epoc

In [58]:
# Evaluasi model pada data uji
loss, accuracy = model.evaluate(x_test, y_test)
print("Loss on test set:", loss)
print("Accuracy on test set:", accuracy)


115/115 [==============================] - 1s 5ms/step - loss: 0.2254 - accuracy: 0.9130
Loss on test set: 0.22543779015541077
Accuracy on test set: 0.9130434989929199


In [59]:
from sklearn.metrics import confusion_matrix, accuracy_score
# Membuat prediksi pada data uji
predictions = model.predict(x_test)

# Mengubah prediksi menjadi label kelas (0 atau 1)
predicted_labels = (predictions > 0.5).astype(int)

# Membuat confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_labels)

# Menghitung akurasi
accuracy = accuracy_score(y_test, predicted_labels)

print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)


115/115 [==============================] - 1s 3ms/step
Confusion Matrix:
[[1941  141]
 [ 177 1398]]
Accuracy: 0.9130434782608695
